# RSA

## references
- [generate prime](https://medium.com/@prudywsh/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb)
- [mod inverse](https://stackoverflow.com/questions/4798654/modular-multiplicative-inverse-function-in-python)
- [encode string as int](https://stackoverflow.com/questions/12625627/python3-convert-unicode-string-to-int-representation)

In [463]:
""" HELPER METHODS """

import math, sys
# Use randrange for is_prime test
from random import randrange
# Use secrets library for number generation
from secrets import randbits, randbelow
# euclidian algorithm

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# modular multiplicative inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
                                                                                                                                                                                            

# This code is contributed by Nikita Tiwari. 
def is_prime(n, k=128):
    """ Test if a number is prime        
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do        
    
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False   
            return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly        
        Args:
            length -- int -- the length of the number to generate, in bits        
            
        return a integer
    """
    # generate random bits
    p = randbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1    
    
    return p 

def generate_prime_number(length=1024):
    """ Generate a prime        Args:
            length -- int -- length of the prime to generate, in bits return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p


In [464]:
import binascii, sys

class RSA:
    """ 
        RSA class with encryption and decryption methods
    """
    def __init__(self):
        pass
    
    def encrypt(self, message, pub):
        print("message: ", message)
        m = self.string_to_int(message)
        block_size = None
        assert m < pub["n"]
        print("m: ", m)
        cipher = pow(m, pub["e"], pub["n"])
        print("c: " , cipher, "\n")
        return cipher
    
    def decrypt(self, c, d, pub):
        # m = c^d mod n
        assert c < pub["n"]
        plaintext = pow(c, d, pub["n"])
        return self.int_to_string(plaintext)
        
    def string_to_int(self, s):
        # https://stackoverflow.com/questions/12625627/python3-convert-unicode-string-to-int-representation

        # encode as utf-8, convert bytes to hexcodes as bytestring...
        # and convert to int specifying base 16(hex)
        i  = int(binascii.hexlify(s.encode('utf-8')), 16)
        #print(i)
        return i
    
    def int_to_string(self, i):
        # convert int to hex, unhexlify and decode
        return binascii.unhexlify(hex(i)[2:]).decode('utf-8')

    def generateKeys(self, keysize=):
        # generate two large primes p and q (each approx 100 digits)
        p = generate_prime_number(keysize)
        #print(len(str(p)))
        q = generate_prime_number(keysize)
        #print(len(str(q)))
        # computer n = p*q
        n = p*q
        # r = (p − 1) ∗ (q − 1)
        r = (p - 1) * (q - 1)
        print("SIZE OF r in bits", sys.getsizeof(r)*8)
        # choose large prime e: 1 < e < r
        e = randbelow(r)
        e = 11
        #Use Euclid's Algorithm to verify that e and phi(n) are comprime                                                                                          
        g = math.gcd(e, r)                                                                                                                                           
        while g != 1:                                                                                                                                             
            e = randbelow(r)                                                                                                                          
            g = math.gcd(e, r) 

        d = modinv(e, r)
#         print("Size of d in bits", sys.getsizeof(d)*8)
        # keeps d private publish pair(e, n) {public key
        public = {"e" : e, "n": n}
        private = d
        return (public, private)

    
rsa = RSA()

public, private = rsa.generateKeys()
print("Public keys\n", "e: ", public["e"],",","n: ", public["n"])
print("\nPrivate key\nd: ", private, "\n\n")

%time c = rsa.encrypt("Hey hows it going you big old fart hows it going you big old fart", public)
%time m = rsa.decrypt(c, private, public)
print(m)

SIZE OF r in bits 2048
Public keys
 e:  32928492546862006224703130732362847934653090620461218533355915593362884227742874306999117241886284001396553822061604737477353463097356818778192005048918953110627247147102793472581506917433867930892975272456643025057130526158017796387877448709885598520659900457896837160851332856744217299307927038324968086935208366125327080135687288952192386717682062465502965124446996698890059702326426509497715232158348232334610938534763456633232888949548132766538097737435673082508422193007351770676817372368021200163894121173007956896707 , n:  509471551081924850400148012198287521419894540517124601878524131796917860161756257642853539635647670111988616096367962542700985814941549533027957155015288587390416962620240269589927654475071204577657036970608158616463128447647937248115015351399795438217395117509580019464260195363590327079172425740062655504887571873956132669681315474350031025872506675320916855593141487958850014151979934921052240271759228600176519749927547408122